<a href="https://colab.research.google.com/github/15nisha/Tag_prediction/blob/main/final_tag_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [162]:
import pandas as pd
import numpy as np
import csv 


import matplotlib.pyplot as plt
import seaborn as sns

import warnings

import pickle
import time

import re
from bs4 import BeautifulSoup

import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation

from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import hamming_loss
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

warnings.filterwarnings("ignore")

**importing file**

In [163]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [164]:
data = pd.read_csv("/content/drive/MyDrive/tag_recommendation/data.csv")

**Data Cleaning**

In [165]:
# load stop words
# import nltk
nltk.download('stopwords')
nltk.download('wordnet')
stop_word = stopwords.words('english')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [166]:
def cleanig_data(dataset):
  ''' function to clean text data . It will remove all special char , HTML tags and null values'''
  # removing 'Unnamed: 0' , which is present in dataset bcz of no use.
  dataset.drop(columns=['Unnamed: 0'], inplace=True)
  # removing nan values
  dataset = dataset.dropna()
  # changing all description text into lower form
  dataset['description'] = dataset['description'].str.lower()
  # changing all tags text into lower form
  dataset['tags'] = dataset['tags'].str.lower()

  def cleanHtml(sentence):
    ''' It cleans all HTML tags '''
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext
  def cleanPunc(sentence): 
    '''function to clean the word of any punctuation or special characters'''
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned
  def keepAlpha(sentence):
    ''' only keeping words with no special character'''
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent
  
  token=ToktokTokenizer()
  lemma=WordNetLemmatizer()

  def lemitizeWords(text):
    ''' function to apply lemmatizer on text'''
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    return ' '.join(map(str, listLemma))
  # applying all defined fuction on dataset columns
  dataset['description'] = dataset['description'].apply(cleanHtml)
  dataset['description'] = dataset['description'].apply(cleanPunc)
  dataset['description'] = dataset['description'].apply(keepAlpha)  
  dataset['description'] = dataset['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
  dataset['description'] = dataset['description'].apply(lambda x: lemitizeWords(x))
  # returning cleaned dataset
  return dataset

In [167]:
# calling function on uncleaned dataset
data_result = cleanig_data(data)


**Tag Processing**

In [168]:
# split tags in space
data_result['new_tags'] = data_result["tags"].apply(lambda x: x.split())

In [169]:
# add all tags
all_tags = [item for sublist in data_result['new_tags'].values for item in sublist]

In [170]:
# copying cleaned dataset into cleaned_data dataframe
cleaned_data = data_result.copy()

In [171]:
# printing some part of dataset
cleaned_data.head()

,tags,description,new_tags
0,licence-needed supervising-job 5-plus-years-ex...,company employer midstream service provider on...,"[licence-needed, supervising-job, 5-plus-years..."
1,2-4-years-experience-needed salary full-time-job,icr staff accept resume industrial maintenance...,"[2-4-years-experience-needed, salary, full-tim..."
2,part-time-job,great position right person healthcareseeker c...,[part-time-job]
3,licence-needed,large multi specialty health center expand adu...,[licence-needed]
4,5-plus-years-experience-needed full-time-job b...,job purpose account director responsible manag...,"[5-plus-years-experience-needed, full-time-job..."


In [172]:
cleaned_data.shape

(3504, 3)

**train - test Split**

In [173]:
# Defining X and y
X = cleaned_data['description']
y = cleaned_data['new_tags']

**Encoding**

In [174]:
# encoding MultiLabelBinarizer for dependent lables
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html
mulitlabel_binarizer = MultiLabelBinarizer()
y_bin = mulitlabel_binarizer.fit_transform(y)

In [175]:
# printin number of classes in y or tags
classes = mulitlabel_binarizer.classes_

In [176]:
classes

array(['1-year-experience-needed', '2-4-years-experience-needed',
       '5-plus-years-experience-needed', 'associate-needed',
       'bs-degree-needed', 'full-time-job', 'hourly-wage',
       'licence-needed', 'ms-or-phd-needed', 'part-time-job', 'salary',
       'supervising-job'], dtype=object)

In [177]:
# encoding independent labels or descriptions
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
vectorizer = TfidfVectorizer(analyzer = 'word',
                                       min_df=0.0,
                                       max_df = 1.0,
                                       strip_accents = None,
                                       encoding = 'utf-8', 
                                       preprocessor=None,
                                       max_features=1000)

multilabel_x_data = vectorizer.fit_transform(X)

In [178]:
# splitting encoded x and y into X_train ,X_test,y_test,y_train with train-test ratio of 70:30
X_train, X_test, y_train, y_test = train_test_split(multilabel_x_data, y_bin, test_size=0.3,random_state=123)

In [179]:
# printing shape of X-train , X-test ,y-train,y-test
print("Dimensions of train data X:",X_train.shape, "Y :",y_train.shape)
print("Dimensions of test data X:",X_test.shape,"Y:",y_test.shape)

Dimensions of train data X: (2452, 1000) Y : (2452, 12)
Dimensions of test data X: (1052, 1000) Y: (1052, 12)


**Training Model with logistic regression**

In [180]:
# now taining data with Onevsrest classifier technique using logistic regression algorithm
classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.00001, penalty='l1'))
# fitting X_train,y_train on classifier defined above
classifier.fit(X_train,y_train )
# predicting y value by giving X_test as input
predictions = classifier.predict (X_test)
# predicting y value by giving X_train as input
predictions_train = classifier.predict (X_train)

In [181]:
# assigning y_test to y_true
y_true = y_test
# assigning predicted y value of X_test to y_logits
y_logits = predictions

In [182]:
#  '''it is computing fraction of equality of two array'''
def result_check(x, y):
  count = 0
  for i in range(len(x)):
    if x[i] == y[i]:
      count += 1
  result = count / len(x)
  return result

In [183]:
# https://stackoverflow.com/questions/46799261/how-to-create-an-exact-match-eval-metric-op-for-tensorflow
def subset_accuracy(y_true,y_predict,threshold):
  ''' it is computing accuracy of model on the basis of , equality fraction between true values and prediction values '''
  count = 0
  for j in range(len(y_true)):
    if result_check(y_true[j] , y_predict[j]) >= threshold:
      count = count + 1
  accuracy = count / len(y_true)
  return accuracy


In [184]:
# giving some set of equality fraction and findin accuracy
threshold_list = [1,.9,.8,.7,.6,.5]
for i in threshold_list:
  print(f'for threshold {i} accuracy is = ',subset_accuracy(y_true,y_logits,threshold=i))

for threshold 1 accuracy is =  0.12167300380228137
for threshold 0.9 accuracy is =  0.40209125475285173
for threshold 0.8 accuracy is =  0.6739543726235742
for threshold 0.7 accuracy is =  0.8830798479087453
for threshold 0.6 accuracy is =  0.9771863117870723
for threshold 0.5 accuracy is =  0.9980988593155894


In [185]:
# printing overall train accuracy of model 
print("train Accuracy :",metrics.accuracy_score(y_train, predictions_train))
# printing overall test accuracy of model
print("test Accuracy :",metrics.accuracy_score(y_test, predictions))
# printing hamming loss of  test data
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))

# printing micro precision of test data
precision = precision_score(y_test, predictions, average='micro')
# printing micro recall of test data
recall = recall_score(y_test, predictions, average='micro')
# printing micro f1-score of test data
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
# printing macro precision of test data
precision = precision_score(y_test, predictions, average='macro')
# printing macro recall of test data
recall = recall_score(y_test, predictions, average='macro')
# printing macro f1-score of test data
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

train Accuracy : 0.9155791190864601
test Accuracy : 0.12167300380228137
Hamming loss  0.1623891001267427
Micro-average quality numbers
Precision: 0.4998, Recall: 0.5051, F1-measure: 0.5024
Macro-average quality numbers
Precision: 0.4776, Recall: 0.4479, F1-measure: 0.4535


## saving model##

In [186]:
# saving the model using joblib
joblib.dump(classifier , 'model.pkl')
joblib.dump(vectorizer , 'vectorizer.pkl')

['vectorizer.pkl']

**Loading Model**

In [187]:
# to map index with class name
def print_class(arr):
  cls_list = []
  for index, value in enumerate(arr):
    if value == 1:
      cls_list.append(classes[index])
  return cls_list
   

In [188]:
# loading saved model
# saving vectorizer as vect
vect = joblib.load('vectorizer.pkl')
# saving classifier as model
model = joblib.load('model.pkl')

In [189]:
# testing = 'any input text which is to be texted'

In [190]:
# first encode input same as in trained model
# v = vect.transform([testing])
# predict with classifiers
# p = model.predict(v)
# mapping predicted value with classes
# testing_result=print_class(p[0])

**Storing Result**

In [191]:
def simlr(testing_result,true):
  ''' it will compute how similar two lists are'''
  count = 0
  for i in true:
    if i in testing_result:
      count= count+1
  return count/len(true)

In [192]:
# defininf data with four columns , initialised  with empty list
d = {
    "Input":[],
     "Prediction":[],
     "true_result":[],
     "similarity":[],

}

In [193]:
# giving a rainge of values of i which will select that corresponding description and tags
for i in range(3465,3473):
  testing = data['description'][i]
# encodig input text
  v = vect.transform([testing])
# predicting model
  p = model.predict(v)
  # mappin result with classes
  testing_result=print_class(p[0])

  # retriving real tags from dataset and splitting them bcz they string
  true = data['tags'][i].split()

# computing similarity between real tags and predicted tags
  sm = simlr(testing_result,true)

# appending calculated data in corresponding colun
  d['Input'].append(testing)
  d['Prediction'].append(testing_result)
  d['true_result'].append(true)
  d['similarity'].append(sm)

In [194]:
# making dataframe
x = pd.DataFrame(d)

In [195]:
x

,Input,Prediction,true_result,similarity
0,"We are seeking a fleet mechanic to keep autos,...","[2-4-years-experience-needed, 5-plus-years-exp...","[2-4-years-experience-needed, part-time-job]",0.500000
1,We are recruiting dynamic professionals like y...,[],[salary],0.000000
2,Pharmaceutical Formulation Chemist Responsib...,"[2-4-years-experience-needed, bs-degree-needed]","[bs-degree-needed, 1-year-experience-needed]",0.500000
3,Please email resume and/or cover letter first....,[],[supervising-job],0.000000
4,A partially paralyzed man in Madison requires ...,"[part-time-job, salary]",[part-time-job],1.000000
5,We are looking for a PT Driver for our Fixed R...,"[full-time-job, hourly-wage, part-time-job]","[hourly-wage, licence-needed, part-time-job]",0.666667
6,Registered Nurses needed for positions in Dall...,"[2-4-years-experience-needed, licence-needed]",[licence-needed],1.000000
7,Registered Nurses needed for *acute care* posi...,"[2-4-years-experience-needed, licence-needed]","[2-4-years-experience-needed, licence-needed, ...",0.666667


In [196]:
# storing dataframe into csv file named result.csv
x.to_csv('result.csv',index=False)